# TCGA_Hallmarks_NES_calculation

Script related to figure s3g

### Aim:
- Calculate TCGA hallmarks NES

In [1]:
sessionInfo()

R version 3.5.0 (2018-04-23)
Platform: x86_64-apple-darwin15.6.0 (64-bit)
Running under: macOS High Sierra 10.13.6

Matrix products: default
BLAS: /Library/Frameworks/R.framework/Versions/3.5/Resources/lib/libRblas.0.dylib
LAPACK: /Library/Frameworks/R.framework/Versions/3.5/Resources/lib/libRlapack.dylib

locale:
[1] C/UTF-8/C/C/C/C

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

loaded via a namespace (and not attached):
 [1] compiler_3.5.0  IRdisplay_0.6.1 pbdZMQ_0.3-3    tools_3.5.0    
 [5] htmltools_0.3.6 base64enc_0.1-3 crayon_1.3.4    Rcpp_1.0.1     
 [9] uuid_0.1-2      IRkernel_0.8.14 jsonlite_1.6    digest_0.6.18  
[13] repr_0.17       evaluate_0.13  

## Load packages

In [1]:
library(GSVA)
library(BBmisc)
library(nortest)

Warning message:
“package ‘GSVA’ was built under R version 3.5.1”
Attaching package: ‘BBmisc’

The following object is masked from ‘package:base’:

    isFALSE



In [2]:
setwd("~/git_repos/HGSOC_TME_Heterogeneity/Scripts/3/")

In [3]:
data <- read.csv('../../Data/1/TCGAov_RNAseq.txt',
                 sep='\t',
                 header=T,
                 row.names='GeneSymbol')

In [4]:
head(data)

,TCGA.04.1348.01,TCGA.04.1357.01,TCGA.04.1362.01,TCGA.04.1364.01,TCGA.04.1365.01,TCGA.04.1514.01,TCGA.04.1519.01,TCGA.09.0364.01,TCGA.09.0366.01,TCGA.09.0367.01,⋯,TCGA.61.2102.01,TCGA.61.2104.01,TCGA.61.2109.01,TCGA.61.2110.01,TCGA.61.2111.01,TCGA.61.2113.01,TCGA.OY.A56P.01,TCGA.OY.A56Q.01,TCGA.VG.A8LO.01,TCGA.WR.A838.01
A1BG,66.4695,65.5664,41.6412,187.0368,23.9295,32.8123,46.3418,162.8522,88.5518,192.7354,⋯,63.7012,99.1485,38.3684,140.3668,94.7103,70.4394,43.1736,33.6854,27.9604,16.2092
A1BG-AS1,36.3243,28.1315,23.2465,114.6008,10.4957,20.1900,17.2007,83.0906,65.6758,55.1265,⋯,30.3602,71.1797,26.0239,77.2415,61.4890,106.1623,47.8864,25.3482,17.9853,42.9450
A1CF,0.0000,0.0000,0.3310,0.0000,0.0000,0.0000,0.0000,0.9034,0.0000,0.0000,⋯,0.0000,0.0000,0.0000,1.9057,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
A2M,5899.8279,9384.4401,3350.4207,1455.2316,3999.3792,3224.5797,1986.1593,1487.3247,10557.8325,4603.9013,⋯,10264.1373,7220.7664,7200.1144,1819.9428,6179.7358,5365.7972,8773.5936,10006.7877,1888.1746,4205.6055
A2M-AS1,118.4566,111.0026,71.3613,67.8607,52.1501,224.7582,182.0650,120.2838,53.6419,92.9112,⋯,146.6130,45.8648,64.4638,28.5565,100.6605,23.5625,84.2153,582.3492,59.8084,143.4910
A2ML1,7.5289,54.6875,5.6263,5.0883,3.3148,3.7421,6.2305,278.2486,4.7763,653.6058,⋯,1.7993,4.6124,0.4768,17.1510,15.6109,2.4461,98.3162,269.5768,4.5092,224.5290


In [5]:
data_mtx <- as.matrix(data)

In [6]:
Hallmark_geneSets <- read.csv('../../Data/1/HallmarksStromaImmune_GeneSets.txt',
                              sep='\t',
                              header=F,
                              row.names=1)

In [7]:
head(Hallmark_geneSets)

,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,⋯,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202
HALLMARK_TNFA_SIGNALING_VIA_NFKB,http://www.broadinstitute.org/gsea/msigdb/cards/HALLMARK_TNFA_SIGNALING_VIA_NFKB,JUNB,CXCL2,ATF3,NFKBIA,TNFAIP3,PTGS2,CXCL1,IER3,CD83,⋯,EIF1,BMP2,DUSP4,PDLIM5,ICOSLG,GFPT2,KLF2,TNC,SERPINB8,MXD1
HALLMARK_HYPOXIA,http://www.broadinstitute.org/gsea/msigdb/cards/HALLMARK_HYPOXIA,PGK1,PDK1,GBE1,PFKL,ALDOA,ENO2,PGM1,NDRG1,HK2,⋯,HDLBP,ILVBL,NCAN,TGM2,ETS1,HOXB9,SELENBP1,FOSL2,SULT2B1,TGFB3
HALLMARK_CHOLESTEROL_HOMEOSTASIS,http://www.broadinstitute.org/gsea/msigdb/cards/HALLMARK_CHOLESTEROL_HOMEOSTASIS,FDPS,CYP51A1,IDI1,FDFT1,DHCR7,SQLE,HMGCS1,NSDHL,LSS,⋯,,,,,,,,,,
HALLMARK_MITOTIC_SPINDLE,http://www.broadinstitute.org/gsea/msigdb/cards/HALLMARK_MITOTIC_SPINDLE,ARHGEF2,CLASP1,KIF11,KIF23,ALS2,ARF6,MYO9B,MYH9,TUBGCP3,⋯,RHOT2,SORBS2,CDC42EP1,VCL,CLIP1,STK38L,YWHAE,RAPGEF5,CEP72,CSNK1D
HALLMARK_WNT_BETA_CATENIN_SIGNALING,http://www.broadinstitute.org/gsea/msigdb/cards/HALLMARK_WNT_BETA_CATENIN_SIGNALING,MYC,CTNNB1,JAG2,NOTCH1,DLL1,AXIN2,PSEN2,FZD1,NOTCH4,⋯,,,,,,,,,,
HALLMARK_TGF_BETA_SIGNALING,http://www.broadinstitute.org/gsea/msigdb/cards/HALLMARK_TGF_BETA_SIGNALING,TGFBR1,SMAD7,TGFB1,SMURF2,SMURF1,BMPR2,SKIL,SKI,ACVR1,⋯,,,,,,,,,,


In [8]:
Hallmark_geneSets <- Hallmark_geneSets[-1]

In [9]:
head(Hallmark_geneSets)

,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,⋯,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202
HALLMARK_TNFA_SIGNALING_VIA_NFKB,JUNB,CXCL2,ATF3,NFKBIA,TNFAIP3,PTGS2,CXCL1,IER3,CD83,CCL20,⋯,EIF1,BMP2,DUSP4,PDLIM5,ICOSLG,GFPT2,KLF2,TNC,SERPINB8,MXD1
HALLMARK_HYPOXIA,PGK1,PDK1,GBE1,PFKL,ALDOA,ENO2,PGM1,NDRG1,HK2,ALDOC,⋯,HDLBP,ILVBL,NCAN,TGM2,ETS1,HOXB9,SELENBP1,FOSL2,SULT2B1,TGFB3
HALLMARK_CHOLESTEROL_HOMEOSTASIS,FDPS,CYP51A1,IDI1,FDFT1,DHCR7,SQLE,HMGCS1,NSDHL,LSS,MVD,⋯,,,,,,,,,,
HALLMARK_MITOTIC_SPINDLE,ARHGEF2,CLASP1,KIF11,KIF23,ALS2,ARF6,MYO9B,MYH9,TUBGCP3,CKAP5,⋯,RHOT2,SORBS2,CDC42EP1,VCL,CLIP1,STK38L,YWHAE,RAPGEF5,CEP72,CSNK1D
HALLMARK_WNT_BETA_CATENIN_SIGNALING,MYC,CTNNB1,JAG2,NOTCH1,DLL1,AXIN2,PSEN2,FZD1,NOTCH4,LEF1,⋯,,,,,,,,,,
HALLMARK_TGF_BETA_SIGNALING,TGFBR1,SMAD7,TGFB1,SMURF2,SMURF1,BMPR2,SKIL,SKI,ACVR1,PMEPA1,⋯,,,,,,,,,,


In [10]:
Hallmark_geneSets <- convertRowsToList(Hallmark_geneSets)

In [11]:
hsi_NES <- gsva(data_mtx,
                Hallmark_geneSets,
                method='ssgsea',
                min.sz=0,
                max.sz=1000,
                ssgsea.norm=T)

Warning message in .local(expr, gset.idx.list, ...):
“338 genes with constant expression values throuhgout the samples.”

Estimating ssGSEA scores for 52 gene sets.
  |                                                                      |   0%Using parallel with 4 cores
  |======================================================================| 100%


In [12]:
hsi_NES <- data.frame("Term"=rownames(hsi_NES),
                      hsi_NES)

rownames(hsi_NES) <- NULL

In [13]:
head(hsi_NES)

Term,TCGA.04.1348.01,TCGA.04.1357.01,TCGA.04.1362.01,TCGA.04.1364.01,TCGA.04.1365.01,TCGA.04.1514.01,TCGA.04.1519.01,TCGA.09.0364.01,TCGA.09.0366.01,⋯,TCGA.61.2102.01,TCGA.61.2104.01,TCGA.61.2109.01,TCGA.61.2110.01,TCGA.61.2111.01,TCGA.61.2113.01,TCGA.OY.A56P.01,TCGA.OY.A56Q.01,TCGA.VG.A8LO.01,TCGA.WR.A838.01
HALLMARK_TNFA_SIGNALING_VIA_NFKB,0.4135076,0.3815939,0.3926226,0.2323723,0.3758730,0.2598795,0.2713037,0.1626862,0.4179557,⋯,0.3120871,0.4260576,0.3664718,0.2828402,0.3253221,0.4657757,0.2317061,0.1541163,0.3250998,0.2707500
HALLMARK_HYPOXIA,0.3853903,0.3595335,0.3717941,0.3447506,0.3980746,0.3719009,0.3461878,0.2898088,0.4013578,⋯,0.4071586,0.3606321,0.3947787,0.3657881,0.3757698,0.4593939,0.3415513,0.2912492,0.3526980,0.3406164
HALLMARK_CHOLESTEROL_HOMEOSTASIS,0.4770949,0.4269293,0.4462245,0.4662101,0.4731517,0.4151925,0.4239398,0.4261344,0.4259422,⋯,0.4176919,0.4799393,0.4549610,0.4044108,0.4395699,0.4665223,0.3567554,0.3503447,0.4510203,0.3984776
HALLMARK_MITOTIC_SPINDLE,0.5344019,0.4672820,0.5133319,0.5496255,0.5434503,0.5471210,0.4704768,0.5515066,0.4570623,⋯,0.5522846,0.4963472,0.5353623,0.4995424,0.5317816,0.5784247,0.5682453,0.5248388,0.5387706,0.5551333
HALLMARK_WNT_BETA_CATENIN_SIGNALING,0.2775478,0.2940325,0.3067881,0.3415397,0.3517504,0.4276304,0.3563603,0.4140771,0.3686600,⋯,0.4445251,0.3795205,0.3370293,0.3447636,0.3727482,0.3895140,0.4175961,0.4287844,0.3232789,0.3987617
HALLMARK_TGF_BETA_SIGNALING,0.5004228,0.5412242,0.5573136,0.5259441,0.5198802,0.5822179,0.5121674,0.4967842,0.5610706,⋯,0.5991747,0.5365324,0.5238909,0.5056704,0.5829953,0.6275223,0.5476635,0.5235076,0.5202442,0.5095698


In [14]:
write.table(hsi_NES,
            file='../../Data/3/HallmarksStromaImmune_NES_TCGAOV.txt',
            sep='\t',
            row.names=F,
            col.names=T)

# End script